In [1]:
from os import path
import pandas as pd
import glob

In [2]:
NSCLC_dir = '/src/dermosxai/data/NSCLC'
attr_path = path.join(NSCLC_dir, 'AIM_files_updated-11-10-2020')
xml_paths = glob.glob(path.join(attr_path, '*.xml'))

image_dir = '/lung_cts' # original images downloaded from https://wiki.cancerimagingarchive.net/display/Public/NSCLC-Radiomics

## Read data

In [245]:
import xml.etree.ElementTree as ET

# xml namespaces
ns = {'default': 'gme://caCORE.caCORE/4.4/edu.northwestern.radiology.AIM', 
      'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
      'xsi': 'http://www.w3.org/2001/XMLSchema-instance'}

def get_typecode_value(typeCode_elem):
    if 'codeSystem' in typeCode_elem.attrib:
        value = typeCode_elem.attrib['codeSystem']
    else:
        value = typeCode_elem.find('{uri:iso.org:21090}displayName').attrib['value']
    
    return value

def parse_xml(xml_filename):
    """ Read all the important info from an xml."""
    elements = []
    
    # Parse XML
    tree = ET.parse(xml_filename)
    root = tree.getroot()
           
    # Read person id
    ids = root.findall('default:person/default:id', ns)
    assert len(ids) == 1
    patient_id = ids[0].attrib['value']
    
    # Read image properties (to identify which image out of the whole study the lesion is on)
    annotations = root.findall('default:imageAnnotations/default:ImageAnnotation', ns)
    assert len(annotations) == 1
    annotation = annotations[0]
    image_name = annotation.find('default:name', ns).attrib['value']
    image_comment = annotation.find('default:comment', ns).attrib.get('value', '')
    
    # Get lesion properties (nvm, always the same)
    lesions = annotation.findall('default:imagingObservationEntityCollection/default:ImagingObservationEntity', ns)
    if len(lesions) > 1:
        feature_sets = [lesion.find('default:imagingObservationCharacteristicCollection', ns) for lesion in lesions]
        has_features = [fs is not None for fs in feature_sets]
        if sum(has_features) > 2:
            raise ValueError('More than one lesion has features')
        lesion = [l for l, hf in zip(lesions, has_features) if hf][0]
    else:
        lesion = lesions[0]
#     lesion_label = lesion.find('default:label', ns).attrib['value']
#     lesion_valuecode = lesion.find('default:typeCode', ns).attrib['code'] # ?
#     lesion_value = get_typecode_value(lesion.find('default:typeCode', ns))
#     lesion_comment = lesion.find('default:comment', ns).attrib.get('value', '')
    
    # Get lesion annotations
    feature_sets = lesion.findall('default:imagingObservationCharacteristicCollection', ns)
    assert len(feature_sets) == 1
    feature_set = feature_sets[0]
    
    feature_labels = []
    feature_valuecodes = []
    feature_values = []
    for feature in feature_set:    
        feature_labels.append(feature.find('default:label', ns).attrib['value'])
        feature_valuecodes.append(feature.find('default:typeCode', ns).attrib['code'])
        feature_values.append(get_typecode_value(feature.find('default:typeCode', ns)))
        
    # Get centroid of lesion
    entities = annotation.findall('default:markupEntityCollection/default:MarkupEntity', ns)
    # assert len(entities) == 1 # only one patient R01-136 has two, they both point to the same thing
    entity = entities[0]
    assert entity.attrib["{{{xsi}}}type".format(**ns)] == 'TwoDimensionCircle'
    coords = entity.findall('default:twoDimensionSpatialCoordinateCollection/default:TwoDimensionSpatialCoordinate', ns)
    assert len(coords) == 2
    x1, x2 = [coord.find('default:x', ns).attrib['value'] for coord in coords]
    y1, y2 = [coord.find('default:y', ns).attrib['value'] for coord in coords]
    
    # Get image info
    studies = annotation.findall('default:imageReferenceEntityCollection/default:ImageReferenceEntity/default:imageStudy', ns)
    assert len(studies) == 1 # only one patient R01-136 has two, they both point to the same thing
    study = studies[0]
    study_id = study.find('default:instanceUid', ns).attrib['root']
    series = study.findall('default:imageSeries', ns)
    assert len(series) == 1
    serie = series[0]
    series_id = serie.find('default:instanceUid', ns).attrib['root']
    
    return (patient_id, image_name, image_comment, feature_labels, feature_valuecodes, 
            feature_values, x1, x2, y1, y2, study_id, series_id)

In [262]:
parsed_features = [parse_xml(xml) for xml in xml_paths]
column_names = ['patient_id', 'image_name', 'image_comment', 'feature_labels',
                'feature_valuecodes', 'feature_values', 'lesion_x1', 'lesion_x2', 
                'lesion_y1', 'lesion_y2', 'study_id', 'series_id']
df = pd.DataFrame(parsed_features, columns=column_names)

In [263]:
# Clean up data
df['image_name'] = df['image_name'].str.replace('~sp1~-~sp1~-1~sp1~#FFFFFF', '')

df['lesion_x'] = (df['lesion_x1'].astype(float) + df['lesion_x2'].astype(float)) / 2
df['lesion_y'] = (df['lesion_y1'].astype(float) + df['lesion_y2'].astype(float)) / 2
df = df.drop(labels=['lesion_x1', 'lesion_x2', 'lesion_y1', 'lesion_y2'], axis='columns')

modality, comment, z  = zip(*df['image_comment'].str.split(' / '))
# df['modality'] = modality # always CT
df['image_comment'] = description
df['lesion_z'] = pd.to_numeric(z)

### Create attributes dset

In [72]:
# Find features that have more than one entry per lesion (multi-label)
repeated = []
for l in df['feature_labels']:
    repeated_in_l = [a for a in set(l) if l.count(a) > 1]
    repeated.extend(repeated_in_l)
repeated_features = set(repeated)
print(repeated_features) # other than these three, all others can become its own column (single valued features)

{'Lung Parencyma Features', 'Nodule Internal Features', 'Nodule Associated Findings'}


In [73]:
rows = []
for idx, r in df.iterrows():
    row = {'patient_id': r['patient_id']}
    for lbl, value in  zip(r['feature_labels'], r['feature_values']):       
        if lbl in repeated_features:
            row[lbl+'$'+value] = True
        else:
            row[lbl] = value
    rows.append(row)

In [74]:
features = pd.DataFrame(rows)
features.columns = features.columns.str.lower().str.replace(' ', '_')

# change NaNs in the absent/present stuff to False
col_filter = [col for col in features if '$' in col]
features[col_filter] = features[col_filter].fillna(False)

In [75]:
features

,patient_id,axial_location,nodule_attenuation,nodule_margins-primary_pattern,nodule_margins-secondary_pattern,nodule_shape,nodule_calcification,nodule_associated_findings$attachment_to_pleura,nodule_associated_findings$vascular_convergence,nodule_periphery,...,nodule_internal_features$necrosis,nodule_internal_features$cavitated,lung_parencyma_features$airway_ectasia,anatomic_fibrosis_distribution,axial_fibrosis_distribution,fibrosis_type,nodule_internal_features$nodule_cysts,lung_parencyma_features$mosaic_oligemia,lung_parencyma_features$tree-in-bud_sign,nodule_associated_findings$attachment_to_bronchus
0,R01-004,peripheral,solid,lobulated,spiculated,oval,none,True,True,emphysema,...,False,False,False,NaN,NaN,NaN,False,False,False,False
1,R01-126,peripheral,partially solid,spiculated,irregular,complex,none,False,True,emphysema,...,False,False,False,NaN,NaN,NaN,False,False,False,False
2,R01-078,peripheral,partially solid,poorly defined,irregular,complex,none,True,False,emphysema,...,False,False,False,NaN,NaN,NaN,False,False,False,False
3,R01-107,peripheral,partially solid,poorly defined,lobulated,complex,none,True,False,normal,...,False,False,False,NaN,NaN,NaN,False,False,False,False
4,R01-039,central,solid,irregular,lobulated,oval,none,False,False,emphysema,...,False,False,False,NaN,NaN,NaN,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,R01-091,peripheral,partially solid,spiculated,irregular,complex,none,False,True,emphysema,...,False,False,False,NaN,NaN,NaN,False,False,False,False
186,R01-033,peripheral,solid,spiculated,lobulated,round,none,True,False,normal,...,False,False,False,NaN,NaN,NaN,False,False,False,False
187,R01-075,peripheral,solid,spiculated,lobulated,complex,peripheral,False,True,normal,...,False,False,False,NaN,NaN,NaN,False,False,False,False
188,AMC-003,peripheral,partially solid,irregular,smooth,complex,none,True,False,normal,...,False,False,False,NaN,NaN,NaN,False,False,False,False


In [76]:
features.isna().any()

patient_id                                                                  False
axial_location                                                              False
nodule_attenuation                                                          False
nodule_margins-primary_pattern                                              False
nodule_margins-secondary_pattern                                            False
nodule_shape                                                                False
nodule_calcification                                                        False
nodule_associated_findings$attachment_to_pleura                             False
nodule_associated_findings$vascular_convergence                             False
nodule_periphery                                                            False
satellite_nodules_in_primary_lesion_lobe_(greater_than_4mm_noncalcified)    False
nodules_in_non-lesion_lobe_same_lung_(greater_than_4mm_noncalcified)        False
nodules_in_contr

In [77]:
joint_df = pd.merge(df, features, on='patient_id')
joint_df = joint_df.drop(['feature_labels', 'feature_valuecodes', 'feature_values'], axis='columns')

In [78]:
joint_df

,patient_id,image_name,image_comment,lesion_z,lesion_x,lesion_y,axial_location,nodule_attenuation,nodule_margins-primary_pattern,nodule_margins-secondary_pattern,...,nodule_internal_features$necrosis,nodule_internal_features$cavitated,lung_parencyma_features$airway_ectasia,anatomic_fibrosis_distribution,axial_fibrosis_distribution,fibrosis_type,nodule_internal_features$nodule_cysts,lung_parencyma_features$mosaic_oligemia,lung_parencyma_features$tree-in-bud_sign,nodule_associated_findings$attachment_to_bronchus
0,R01-004,Lesion1,CT / THIN LUNG WINDOW / 54,54,333.819706,186.230608,peripheral,solid,lobulated,spiculated,...,False,False,False,NaN,NaN,NaN,False,False,False,False
1,R01-126,Lesion1,CT / 1.25MM CHEST BONE / 120,1,121.916354,306.093890,peripheral,partially solid,spiculated,irregular,...,False,False,False,NaN,NaN,NaN,False,False,False,False
2,R01-078,Lesion1,CT / SUPER / 50,50,366.495726,307.692308,peripheral,partially solid,poorly defined,irregular,...,False,False,False,NaN,NaN,NaN,False,False,False,False
3,R01-107,Lesion1,CT / 1.25MM CHEST BONE PLUS / 40,40,155.837953,310.038380,peripheral,partially solid,poorly defined,lobulated,...,False,False,False,NaN,NaN,NaN,False,False,False,False
4,R01-039,Lesion1,CT / CT THICK AXIALS 2.5MM / 42,42,179.790356,251.169811,central,solid,irregular,lobulated,...,False,False,False,NaN,NaN,NaN,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,R01-091,Lesion1,CT / LUNG 1MM B45F / 76,76,114.658199,250.120181,peripheral,partially solid,spiculated,irregular,...,False,False,False,NaN,NaN,NaN,False,False,False,False
186,R01-033,Lesion1,CT / THIN LUNG WINDOW / 146,146,376.218029,181.668763,peripheral,solid,spiculated,lobulated,...,False,False,False,NaN,NaN,NaN,False,False,False,False
187,R01-075,Lesion1,CT / THORAX 1.0 B45F / 85,85,339.608511,229.038298,peripheral,solid,spiculated,lobulated,...,False,False,False,NaN,NaN,NaN,False,False,False,False
188,AMC-003,Lesion1,CT / THORAX LUNG 1MM / 123,123,127.193653,345.825478,peripheral,partially solid,irregular,smooth,...,False,False,False,NaN,NaN,NaN,False,False,False,False


### Save

In [ ]:
#df.to_csv(path.join(NSCLC_dir, 'image_info.csv'), index=False)

In [142]:
#TODO: drop feature_valuecodes from df and join
#TODO: Make sure that if no value of the compostive labels xxxx$yyyy is present it should all be nan no all false
#    or should it?

# Create image dataset

## Get paths

In [ ]:
#df = pd.read_csv(path.join(NSCLC_dir, 'lesions.csv')

In [283]:
# Directory where the images for each CT "image" is (a CT image is made of many actual dcm images)
image_paths = image_dir + '/NSCLC Radiogenomics/' + df['patient_id'] + '/' + df['study_id'] + '/' + df['series_id']

In [324]:
tcia_metadata = pd.read_csv(path.join(image_dir, 'metadata.csv'))
def find_segmentations(row):
    segms = tcia_metadata[(tcia_metadata['Subject ID'] == row['patient_id']) & 
                          (tcia_metadata['Modality'] == 'SEG')]
    assert len(segms) <= 1
    if len(segms) == 1:
        rel_path = segms.iloc[0]['File Location']
        return path.join(image_dir, rel_path[2:])

# Directory where the segmentation image is (usually a single image)
segm_paths = df.apply(find_segmentations, axis='columns')

In [328]:
paths = pd.concat([df['patient_id'], image_paths.rename('image_path'), segm_paths.rename('segmentation_path')], axis=1)

In [330]:
paths = paths.dropna()

In [336]:
paths

,patient_id,image_path,segmentation_path
0,R01-004,/lung_cts/NSCLC Radiogenomics/R01-004/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-004/1.3.6.1....
1,R01-126,/lung_cts/NSCLC Radiogenomics/R01-126/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-126/1.3.6.1....
2,R01-078,/lung_cts/NSCLC Radiogenomics/R01-078/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-078/1.3.6.1....
3,R01-107,/lung_cts/NSCLC Radiogenomics/R01-107/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-107/1.3.6.1....
4,R01-039,/lung_cts/NSCLC Radiogenomics/R01-039/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-039/1.3.6.1....
...,...,...,...
184,R01-057,/lung_cts/NSCLC Radiogenomics/R01-057/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-057/1.3.6.1....
185,R01-091,/lung_cts/NSCLC Radiogenomics/R01-091/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-091/1.3.6.1....
186,R01-033,/lung_cts/NSCLC Radiogenomics/R01-033/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-033/1.3.6.1....
187,R01-075,/lung_cts/NSCLC Radiogenomics/R01-075/1.3.6.1....,/lung_cts/NSCLC Radiogenomics/R01-075/1.3.6.1....


## Crop images and resize

In [332]:
from pydicom import dcmread

In [334]:
for segm_path in segm_paths:
    break

In [ ]:
#TODO: Get segmentations and print the average sizes of the 3d-bbox

Note (Michael): I stopped here because we found all lesions are cancers (i.e., no benign lesions/negative class); so we won't use this dset